# Scraper

In [54]:
import pandas as pd 
import requests
import unicodedata
from tqdm import tqdm
from bs4 import BeautifulSoup

def get_links(): 
    response = requests.get('https://en.wikipedia.org/wiki/List_of_language_names')
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    links = [link.get('href') for link in links if (link.get('title') is not None) and ('language' in link.get('title'))]
    links = [link for link in links if link.endswith('_language')]
    links = ["https://en.wikipedia.org"+link for link in links]
    return links

urls = get_links()
print(len(urls))
print(urls[:10])

720
['https://en.wikipedia.org/wiki/Aari_language', 'https://en.wikipedia.org/wiki/Abaza_language', 'https://en.wikipedia.org/wiki/Official_language', 'https://en.wikipedia.org/wiki/Abellen_language', 'https://en.wikipedia.org/wiki/Abenaki_language', 'https://en.wikipedia.org/wiki/Abkhaz_language', 'https://en.wikipedia.org/wiki/Official_language', 'https://en.wikipedia.org/wiki/Abui_language', 'https://en.wikipedia.org/wiki/Abure_language', 'https://en.wikipedia.org/wiki/Acehnese_language']


In [55]:
data = []
errors = []

for url in tqdm(urls): 
    try: 
        lang = url.split('/')[-1]

        response = requests.get(url)
        content  = response.content
        parser   = BeautifulSoup(content, 'html.parser')
        table    = parser.find('table', {'class': 'infobox'})
        regions  = table.find_all('td', {'class': 'infobox-data'})[1].text
        speakers = table.find_all('td', {'class': 'infobox-data'})[3].text

        data.append((lang, regions, speakers))
    except Exception as e: 
        errors.append((e, url))
        continue

df = pd.DataFrame(data, columns=['lang', 'regions', 'speakers'])

  0%|          | 0/720 [00:00<?, ?it/s]

  3%|▎         | 21/720 [00:09<05:03,  2.30it/s]


KeyboardInterrupt: 

In [9]:
urls = urls[0:100]

In [56]:
def find_index_of_headers(url):
    response        = requests.get(url)
    content         = response.content
    parser          = BeautifulSoup(content, 'html.parser')
    table           = parser.find('table', {'class': 'infobox'})
    included_headers = {}
    
    for i in range(0, len(table.find_all('th', {'class': 'infobox-label'}))):
        included_headers[unicodedata.normalize('NFKD', table.find_all('th', {'class': 'infobox-label'}).__getitem__(i).getText())] = i
    return included_headers

In [57]:
urls = 'https://en.wikipedia.org/wiki/Aari_language'

In [59]:
data = []
errors = []

for url in tqdm(urls): 
    # try: 
        lang = url.split('/')[-1]

        print(requests.get(url).status_code)
        response        = requests.get(url)
        content         = response.content
        parser          = BeautifulSoup(content, 'html.parser')
        table           = parser.find('table', {'class': 'infobox'})

        header_indices  = find_index_of_headers(url)

        print(header_indices)

        regions         = table.find_all('td', {'class': 'infobox-data'})[header_indices['Regions']].text
        off_lang        = table.find_all('td', {'class': 'infobox-data'})[header_indices['Official language in']].text
        rec_min_lang    = table.find_all('td', {'class': 'infobox-data'})[header_indices['Recognised minoritylanguage in']].text
        speakers        = table.find_all('td', {'class': 'infobox-data'})[header_indices['Speakers']].text
        family          = table.find_all('td', {'class': 'infobox-data'})[header_indices['Language family']].get_text('title').split('title')
        family          = [x for x in family if x != "\n"]
        dialects        = table.find_all('td', {'class': 'infobox-data'})[header_indices['Dialects']].get_text('title').split('title')
        dialects        = [x for x in dialects if x != "\n"]
        iso3code        = table.find_all('td', {'class': 'infobox-data'})[header_indices['ISO 639-3']].text

        data.append((lang, family, dialects, iso3code, speakers, regions, off_lang, rec_min_lang))
    # except Exception as e: 
    #     errors.append((e, url))
    #     continue

df = pd.DataFrame(data, columns=['lang', 'family', 'dialects', 'iso3code', 'speakers', 'regions', 'off_lang', 'rec_min_lang'])
# pd.DataFrame(family)[0].transform
# print(type(family))
# print(family)
# print(dialects)
# print(iso3code)
# print(type(off_lang))
# print(off_lang)
# print(rec_min_lang)


  0%|          | 0/43 [00:00<?, ?it/s]


MissingSchema: Invalid URL 'h': No scheme supplied. Perhaps you meant https://h?

In [48]:
df

,lang,family,dialects,iso3code,speakers,regions,off_lang,rec_min_lang
